In [2]:
import sys 
import matplotlib.pyplot as plt

pml = pulsedmeasurementlogic
pmal = pulsedmasterlogic
odmr = odmrlogic

import logic.pulsed.pulse_objects as po
import time
import os

In [3]:
class qudi_pulsed_measurements:

    def __init__(self, laser_on = 3e-6, relaxation_pause = 10e-6):

        self.laser_on = laser_on
        self.relaxation_pause = relaxation_pause
        self.pause_total_ns = (self.laser_on + self.relaxation_pause)*1e9
        
        self.max_samples = 2e9
        
        self.trigger_block = self.make_trigger_pulse_block()
    
    def make_next_pulse_block(self):
        '''
        Triggers the next histogram in the fast counter time differences measurement and also give the GUI the laser channel as d_ch8.
        '''
        ele = [] 
        
        d_ch = {'d_ch1': False, 'd_ch2': False, 'd_ch4': False, 'd_ch3': True, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': True}
        ele.append(po.PulseBlockElement(init_length_s=1e-6, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

        pulse_block = po.PulseBlock(name='next_block', element_list=ele)
        
        return pulse_block
    
    def make_trigger_pulse_block(self):
        ele = [] 
        
        d_ch = {'d_ch1': False, 'd_ch2': False, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': True, 'd_ch7': False, 'd_ch8': False}
        ele.append(po.PulseBlockElement(init_length_s=1e-3, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

        pulse_block = po.PulseBlock(name='trigger_block', element_list=ele)
        
        return pulse_block
    
    def make_pulsed_ODMR_pulse_block(self, pi_pulse):
        ele = []
    
        d_ch = {'d_ch1': False, 'd_ch2': False, 'd_ch4': False, 'd_ch3': False, 'd_ch5': True, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
        ele.append(po.PulseBlockElement(init_length_s=pi_pulse*1e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

        d_ch = {'d_ch1': False, 'd_ch2': False, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
        ele.append(po.PulseBlockElement(init_length_s=50e-9, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
        
        d_ch = {'d_ch1': True, 'd_ch2': True, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
        ele.append(po.PulseBlockElement(init_length_s=self.laser_on, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))

        d_ch = {'d_ch1': False, 'd_ch2': False, 'd_ch4': False, 'd_ch3': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}
        ele.append(po.PulseBlockElement(init_length_s=self.relaxation_pause, increment_s=0, pulse_function=None, digital_high=d_ch, laser_on=False))
        
        pulse_block = po.PulseBlock(name=f'pulsed_ODMR_qudi', element_list=ele)
        
        return pulse_block
    
    def make_podmr_ensemble(self, pulse, trig_mw, trig_counter, repetitions, n_freq):
        block_list = []
        for i in range(n_freq):
            block_list.append((trig_mw, 0))
            block_list.append((trig_counter, 0))
            block_list.append((pulse, repetitions))
        
        return block_list
    
    def upload_pulsed_ODMR_ensemble(self, freq_list, pi_pulse, repetitions):
        pulse = self.make_pulsed_ODMR_pulse_block(pi_pulse)
        sequencegeneratorlogic.save_block(pulse)
        
        trig_mw = self.make_trigger_pulse_block()
        sequencegeneratorlogic.save_block(trig_mw)
        
        trig_counter = self.make_next_pulse_block()
        sequencegeneratorlogic.save_block(trig_counter)
        
        bl = self.make_podmr_ensemble(pulse.name, trig_mw.name, trig_counter.name, repetitions, len(freq_list))
        auto_pulse_CW = po.PulseBlockEnsemble(name=f'pulsed_ODMR_qudi', block_list=bl, rotating_frame=False)

        ensemble = auto_pulse_CW
        ensemblename = auto_pulse_CW.name
        sequencegeneratorlogic.save_ensemble(ensemble)
        ret,_,_ = sequencegeneratorlogic.sample_pulse_block_ensemble(ensemblename)
        sequencegeneratorlogic.load_ensemble(ensemblename)

        return ret
    
    def pulsed_ODMR(self,freq_list, pi_pulse, repetitions):
        pulsedmasterlogic.set_ext_microwave_settings(use_ext_microwave=False)
        err = self.upload_pulsed_ODMR_ensemble(freq_list, pi_pulse, repetitions)
        pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                                controlled_variable=freq_list,
                                                number_of_lasers=len(freq_list), 
                                                laser_ignore_list=[], 
                                                alternating=False, 
                                                units=('Hz', 'arb. u.'))
        pulsedmasterlogic.set_fast_counter_settings(record_length = self.laser_on)
        time.sleep(0.5)
        pulsedmasterlogic.set_alternative_data_type('None')
        time.sleep(0.5)

In [14]:
measurement = qudi_pulsed_measurements(laser_on = 4e-6, relaxation_pause = 2e-6)

freq_start=2.86e9
freq_step=500e3
freq_stop=2.88e9

odmrlogic.set_sweep_parameters([freq_start],[freq_stop], [freq_step],-15)
odmrlogic.mw_sweep_on()
time.sleep(1)
odmrlogic.mw_off()
freq_list = odmrlogic.final_freq_list
print(len(freq_list))
print(freq_list)
measurement.pulsed_ODMR(freq_list, pi_pulse=213, repetitions=1000)

41
[2.8600e+09 2.8605e+09 2.8610e+09 2.8615e+09 2.8620e+09 2.8625e+09
 2.8630e+09 2.8635e+09 2.8640e+09 2.8645e+09 2.8650e+09 2.8655e+09
 2.8660e+09 2.8665e+09 2.8670e+09 2.8675e+09 2.8680e+09 2.8685e+09
 2.8690e+09 2.8695e+09 2.8700e+09 2.8705e+09 2.8710e+09 2.8715e+09
 2.8720e+09 2.8725e+09 2.8730e+09 2.8735e+09 2.8740e+09 2.8745e+09
 2.8750e+09 2.8755e+09 2.8760e+09 2.8765e+09 2.8770e+09 2.8775e+09
 2.8780e+09 2.8785e+09 2.8790e+09 2.8795e+09 2.8800e+09]


In [15]:
odmrlogic.mw_sweep_on()

('list', True)

In [17]:
odmrlogic.mw_off()

('list', False)